<a href="https://colab.research.google.com/github/Tranminhtuan48/pythonbasics/blob/main/BaiGiuaKy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [3]:
#Tập dữ liệu Breast Cancer Wisconsin
bcw = pd.read_csv('breast-cancer-wisconsin.data', sep = ",", header=None)

In [4]:
bcw.head(5)

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [5]:
# Hiển thị số dòng, số cột của dữ liệu
bcw.shape

(699, 11)

Ta có các biến của dữ liệu lần lượt là Sample code number(ID), Clump Thickness, Uniformity of Cell Size, Uniformity of Cell Shape, Marginal Adhesion, Single Epithelial Cell Size, Bare Nuclei, Bland Chromatin, Normal Nucleoli, Mitoses và Class (phân lớp)

==> Biến cuối cùng đóng vai trò nhãn lớp.(2.1)

In [6]:
# Kiểm tra các giá trị khác nhau của nhãn lớp
bcw[10].unique()

array([2, 4])

* Biến class có 2 giá trị khác nhau là 2 và 4. Đây là bài toán phân lớp nhị phân (2.2)

* Sample code number (ID) là một thuộc tính không cần thiết trong quá trình huấn luyện (2.3) vì nó chỉ dùng để định danh.

Tiền xử lí dữ liệu

In [7]:
# Xử lí dữ liệu bị lỗi (được biểu diễn bằng "?")
for i in range(11):
    if bcw.loc[bcw[i] == "?"].shape[0] > 0:
      print(i)

6


In [8]:
# Tiến hành xóa các dòng chứa dữ liệu lỗi
bcw = bcw[bcw[6] != "?"]
# Chuyển kiểu dữ liệu từ str về integer
bcw[6] = pd.to_numeric(bcw[6], downcast="integer")

<ipython-input-8-2cbd47ecad05>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bcw[6] = pd.to_numeric(bcw[6], downcast="integer")


In [9]:
for i in range(11):
    if bcw.loc[bcw[i] == "?"].shape[0] > 0:
      print(i)
else:
  print("None ERROR")

None ERROR


In [10]:
# Xác định số lượng các giá trị
bcw[10].value_counts()

2    444
4    239
Name: 10, dtype: int64

In [11]:
#  Chuyển giá trị 2 -> 0 và 4 -> 1 (Cho quá trình phân lớp)
bcw[10] = np.where(bcw[10] == 2, 0, 1)

In [12]:
# Kiểm tra
bcw[10].value_counts()

0    444
1    239
Name: 10, dtype: int64

Phân tích nhãn lớp

* Biến nhãn lớp 2 - 444 quan sát và 4 - 239 quan sát. Ta thấy có sự chênh lệch rõ ràng giữa 2 nhãn lớp. Vậy nhãn lớp không cân bằng (3.1)

* Vì không phải là dữ liệu kiểu thời gian (gắn với chiều thời gian), ta cần xáo trộn dữ liệu khi train_test_split (3.2)

Phân chia train test

In [13]:
# Phân tập dữ liệu thành X () biến độc lập) và y (biến nhãn lớp). Bỏ cột (ID) ở X, lấy y là cột cuối cùng
# Lấy X dưới dạng numpy array
X = bcw.iloc[:, 1:-1].values
y = bcw.iloc[:, -1].values

In [14]:
X.shape

(683, 9)

In [15]:
y.shape

(683,)

In [16]:
# Phân tập dữ liệu thành tập train (80%) và test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=5)

In [17]:
print(f"Shape của X train: {X_train.shape}")
print(f"Shape của y train: {y_train.shape}")
print(f"Shape của X test: {X_test.shape}")
print(f"Shape của y test: {y_test.shape}")

Shape của X train: (546, 9)
Shape của y train: (546,)
Shape của X test: (137, 9)
Shape của y test: (137,)


Huấn luyện mô hình và đánh giá mô hình

In [18]:
# Hàm đánh giá & bảng so sánh
#from sklearn.metrics import accuracy_score
#from sklearn.metrics import precision_score
#from sklearn.metrics import recall_score
#from sklearn.metrics import f1_score
# Hàm tính toán các giá trị đánh giá
def test_score(y_test, y_pred, class_type = "binary"):
  if class_type == "binary":
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    result = [accuracy, precision, recall, f1]
  else:
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average = "macro")
    recall = recall_score(y_test, y_pred, average = "macro")
    f1 = f1_score(y_test, y_pred, average = "macro")
    result = [accuracy, precision, recall, f1]
  return result

In [19]:
# Hàm tạo bảng so sánh các mô hình
def test_table(lr_scores, kNN_scores, nb_scores):
  result = {
      "Scores": ["accuracy", "precision", "recall", "f1"],
      "LogisticRegression": lr_scores,
      "kNN": kNN_scores,
      "NaiveBayes": nb_scores
  }
  return pd.DataFrame(result)

LogisticRegression

**Kết quả 10-fold CV sử dụng cross_val_score(3.3)**

In [20]:
classifier = LogisticRegression(solver='liblinear', max_iter=1500, 
                                multi_class='auto')
scores = cross_val_score(classifier, X_train, y_train, cv=10, 
                       scoring='accuracy')

In [21]:
print(f"Kết quả huấn luyện: {scores}")

Kết quả huấn luyện: [0.94545455 0.90909091 0.98181818 0.98181818 0.98181818 0.96363636
 0.92592593 1.         0.96296296 0.94444444]


Huấn luyện mô hình

In [22]:
# Tạo hàm KfoldCV:
def kFoldCrossValiation(model, X_train, y_train):
    kf = KFold(n_splits=10)
    step = 0
    valitdate_models = {}
    for train2_index, val_index in kf.split(X = X_train,y = y_train):
        step = step + 1
        X_train2, X_val = X_train[train2_index], X_train[val_index]
        y_train2, y_val = y_train[train2_index], y_train[val_index]
        model.fit(X_train2, y_train2)
        y_pred = model.predict(X_val)
        valitdate_models[accuracy_score(y_val, y_pred)] =  model
    max_accuracy = max(list(valitdate_models.keys()))
    return valitdate_models[max_accuracy]

In [23]:
# Huấn luyện mô hình
lr = LogisticRegression(solver='liblinear', max_iter=1500, 
                                multi_class='auto')
lr_optimal = kFoldCrossValiation(lr, X_train, y_train)

In [24]:
# Dự đoán kết quả
y_pred1 = lr_optimal.predict(X_test)

In [25]:
# Tính scores
lr_scores = test_score(y_test, y_pred1)

k-Nearest Neighbor

Kết quả 10-fold CV sử dụng cross_val_score(3.3)

In [26]:
kNN1 = KNeighborsClassifier()
scores_kNN = cross_val_score(kNN1, X_train, y_train, cv=10, 
                       scoring='accuracy')

In [27]:
print(f"Kết quả huấn luyện: {scores_kNN}")

Kết quả huấn luyện: [0.92727273 0.94545455 0.98181818 1.         1.         0.96363636
 0.92592593 1.         0.96296296 0.96296296]


Tìm k tối ưu bằng GridSearchCV

In [28]:
# Xác định k_max
from math import sqrt
m = y_train.shape[0]
k_max = int(sqrt(m)/2)

In [29]:
# Xác định các tham số:
k_values = np.arange(start = 1, stop = k_max + 1, dtype = int)
params = {'n_neighbors': k_values}

In [30]:
# Tìm k tối ưu
kNN = KNeighborsClassifier()
kNN_grid = GridSearchCV(kNN, params, cv=3)
kNN_grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])})

In [31]:
# Dự đoán kết quả:
y_pred2 = kNN_grid.predict(X_test)

In [32]:
# Tính scores
kNN_scores = test_score(y_test, y_pred2)

Naive Bayes

Kết quả 10-fold CV sử dụng cross_val_score(3.3)

In [33]:
gnb_model = GaussianNB()
scores_gnb = cross_val_score(gnb_model, X_train, y_train, cv=10, 
                       scoring='accuracy')

In [34]:
print(f"Kết quả huấn luyện: {scores_gnb}")

Kết quả huấn luyện: [0.92727273 0.92727273 0.98181818 0.96363636 0.98181818 0.96363636
 0.94444444 1.         0.96296296 0.90740741]


* Huấn luyện mô hình

In [35]:
# Huấn luyện mô hình
gnb = GaussianNB()
gnb_optimal = kFoldCrossValiation(gnb, X_train, y_train)

In [36]:
# Dự đoán kết quả:
y_pred3 = gnb_optimal.predict(X_test)

In [37]:
# Tính scores
nb_scores = test_score(y_test, y_pred3)

Đánh giá hiệu năng và so sánh các chỉ số các mô hình (3.4.a & 3.4.b)**

In [38]:
test_table(lr_scores, kNN_scores, nb_scores)

,Scores,LogisticRegression,kNN,NaiveBayes
0,accuracy,0.992701,1.0,0.992701
1,precision,1.000000,1.0,0.979167
2,recall,0.978723,1.0,1.000000
3,f1,0.989247,1.0,0.989474


==> Mô hình kNN có các chỉ số đánh giá tốt hơn so với LogisticRegression và NaiveBayes(3.4.c)**

In [39]:
# Tập dữ liệu Wine
wine = pd.read_csv('wine.data', sep = ",", header=None)

In [40]:
wine.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [41]:
wine.shape

(178, 14)

In [42]:
# Tìm giá trị khác nhau của biến nhãn lớp
wine[0].unique()

array([1, 2, 3])

Ta có các biến của dữ liệu lần lượt là Class (phân lớp), Alcohol, Malic acid, Ash, Alacalinity of ash, Magnesium, Flavonoids, Nonflavanoid phenols, Proanthocyains, Color intensity, Hue, OD280/OD315 of diluted wines và Proline. 

==> Biến đầu tiên đóng vai trò nhãn lớp.(2.1)


Biến class có 3 giá trị khác nhau là 1, 2 và 3 (1 :Barbera, 2: Barolo và 3: Grignolino)

==> Phân lớp đa nhãn (2.2)

Ta giữ lại tất cả các biến khi huấn luyện mô hình (2.3) vì các biến ngoài class đều là các biến liên tục và có khả năng ảnh hưởng đến việc phân lớp

Tiền xử lí dữ liệu

In [43]:
# Xử lí dữ liệu bị lỗi
for i in range(14):
    if wine.loc[wine[i] == "?"].shape[0] > 0:
      print(i)
else:
  print("None ERROR")

None ERROR


Vậy không có giá trị bị lỗi










In [44]:
# Xác định số lượng các giá trị khác nhau
wine[0].value_counts()

2    71
1    59
3    48
Name: 0, dtype: int64

In [45]:
# Chuyển giá trị 1 -> 0, 2 -> 1, 3 -> 2
wine[0].loc[wine[0] == 1] = 0
wine[0].loc[wine[0] == 2] = 1
wine[0].loc[wine[0] == 3] = 2

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [46]:
# Kiểm tra
wine[0].value_counts()

1    71
0    59
2    48
Name: 0, dtype: int64

Phân tích nhãn lớp

* Biến nhãn lớp với nhãn lớp 0 - 59 quan sát, 1 - 71 quan sát và 2 - 48 quan sát. Ta thấy không sự chênh lệch rõ ràng giữa 3 nhãn lớp. Vậy nhãn lớp cân bằng (3.1)

* Vì không phải dữ liệu kiểu thời gian, ta cần tiến hành xáo trộn dữ liệu khi tran_test_split (3.2)

Phân chia train test

In [47]:
# # Phân tập dữ liệu thành X () biến độc lập) và y (biến nhãn lớp). Bỏ cột (ID) ở X, lấy y là cột cuối cùng
# Lấy X dưới dạng numpy array
X2 = wine.iloc[:, 1:].values
y2 = wine.iloc[:, 0].values

In [48]:
# Kiểm tra X:
X2.shape

(178, 13)

In [49]:
# Kiểm tra y:
y2.shape

(178,)

In [50]:
# Phân tập dữ liệu thành tập train (80%) và tập test (20%)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=5)

In [51]:
print(f"Shape của X train: {X_train2.shape}")
print(f"Shape của y train: {y_train2.shape}")
print(f"Shape của X test: {X_test2.shape}")
print(f"Shape của y test: {y_test2.shape}")

Shape của X train: (142, 13)
Shape của y train: (142,)
Shape của X test: (36, 13)
Shape của y test: (36,)


In [52]:
# Tiến hành feature scaling
sc_X = StandardScaler()
X_train2 = sc_X.fit_transform(X_train2)
X_test2 = sc_X.transform(X_test2)

In [53]:
print(f"Kiểm tra X train: \n{X_train2[:2]}\n")
print(f"Kiểm tra X test: \n{X_test2[:2]}")

Kiểm tra X train: 
[[ 0.93714471 -0.61512376 -0.39190501 -0.94695164  1.42911289  0.56919905
   0.92507397 -1.26819521  0.08123484  0.31479467 -0.17219682  0.87573671
   1.03051309]
 [-0.02707735  0.37686901  1.26217226  0.41011025 -0.1973165  -1.16232562
  -1.46746764  1.28053501 -1.43473973 -0.20522444 -0.79322497 -0.38165485
  -0.41820237]]

Kiểm tra X test: 
[[ 1.0918964  -0.41498487  1.62974499 -0.06191127  0.6158982   1.14092889
   0.99485643  0.0061699   0.32518477 -0.24682597  1.19406512  1.15829661
   0.57385278]
 [ 0.18719422 -1.18943537 -2.413555   -1.35997047 -1.52803145  1.1899343
   1.20420382 -0.86995611  1.23128451  0.08598626  0.65584072  0.84748072
  -0.72684175]]


Huấn luyện mô hình và đánh giá mô hình

Logistic Regression

Kết quả 10-fold CV sử dụng cross_val_score (3.3)

In [54]:
wine_classifier = LogisticRegression(solver='liblinear', max_iter=1500, 
                                multi_class='auto')
wine_scores = cross_val_score(wine_classifier, X_train, y_train, cv=10, 
                       scoring='accuracy')

In [55]:
print(f"Kết quả huấn luyện: {wine_scores}")

Kết quả huấn luyện: [0.94545455 0.90909091 0.98181818 0.98181818 0.98181818 0.96363636
 0.92592593 1.         0.96296296 0.94444444]


Huấn luyện mô hình

In [56]:
# Huấn luyện mô hình
wine_lr = LogisticRegression(solver='liblinear', max_iter=1500, 
                                multi_class='auto')
wine_lr_optimal = kFoldCrossValiation(wine_lr, X_train2, y_train2)

In [57]:
# Dự đoán kết quả
y_pred_wine1 = wine_lr_optimal.predict(X_test2)

In [58]:
# Tính scores
wine_lr_scores = test_score(y_test2, y_pred_wine1, class_type = "multy")

Mô hình k-NN

Kết quả 10-fold CV sử dụng cross_val_score (3.3)

In [59]:
wine_kNN1 = KNeighborsClassifier()
wine_scores_kNN = cross_val_score(wine_kNN1, X_train2, y_train2, cv=10, 
                       scoring='accuracy')

In [60]:
print(f"Kết quả huấn luyện: {wine_scores_kNN}")

Kết quả huấn luyện: [1.         0.93333333 0.92857143 1.         1.         1.
 1.         1.         1.         0.85714286]


Tìm giá trị k tối ưu bằng GridSearchCV

In [61]:
# Xác định k_max
from math import sqrt
m2 = y_train2.shape[0]
k_max2 = int(sqrt(m2)/2)

In [62]:
# Xác định các tham số:
k_values2 = np.arange(start = 1, stop = k_max + 1, dtype = int)
params2 = {'n_neighbors': k_values}

In [63]:
# Tìm k tối ưu
kNN2 = KNeighborsClassifier()
kNN_grid2 = GridSearchCV(kNN, params, cv=3)
kNN_grid2.fit(X_train2, y_train2)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])})

In [64]:
# Dự đoán kết quả:
y_pred_wine2 = kNN_grid2.predict(X_test2)

In [65]:
# Tính scores
wine_kNN_scores = test_score(y_test2, y_pred_wine2, class_type = "multy")

Naive Bayes

Kết quả 10-fold CV sử dụng cross_val_score(3.3)

In [66]:
wine_gnb_model = GaussianNB()
wine_scores_gnb = cross_val_score(wine_gnb_model, X_train, y_train, cv=10, 
                       scoring='accuracy')

In [67]:
print(f"Kết quả huấn luyện: {wine_scores_gnb}")

Kết quả huấn luyện: [0.92727273 0.92727273 0.98181818 0.96363636 0.98181818 0.96363636
 0.94444444 1.         0.96296296 0.90740741]


Huấn luyện mô hình

In [68]:
# Huấn luyện mô hình
wine_gnb = GaussianNB()
wine_gnb_optimal = kFoldCrossValiation(wine_gnb, X_train2, y_train2)

In [69]:
# Dự đoán kết quả:
y_pred_wine3 = wine_gnb_optimal.predict(X_test2)

In [70]:
# Tính scores
wine_nb_scores = test_score(y_test2, y_pred_wine3, class_type = "multy")

Bảng đánh giá mô hình

Đánh giá hiệu năng và so sánh các chỉ số của mô hình (3.4.a & 3.4.b)

In [71]:
test_table(wine_lr_scores, wine_kNN_scores, wine_nb_scores)

,Scores,LogisticRegression,kNN,NaiveBayes
0,accuracy,1.0,0.972222,0.916667
1,precision,1.0,0.981481,0.928571
2,recall,1.0,0.969697,0.941176
3,f1,1.0,0.974603,0.927742


Kết luận: Mô hình Logistic Regression có các chỉ số đánh giá tốt hơn so với kNN và Naive Bayes (3.4.c)